In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch import linalg as LA
from torch.utils.data import DataLoader
from transformers import logging
from transformers import pipeline
from tqdm import tqdm

In [2]:
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel

In [3]:
from relation_templates.templates import relations, nominalized_relations, get_templates, get_relation_meta, relations, relation_names, get_relation_cardinality, get_relations_with_last_digit

In [4]:
from TRExData.LamaTRExData import LamaTRExData
from TRExData.AbstractTRExData import AbstractTRExData
from TRExData.HardTRExData import HardKnownTRExData

In [5]:
from FeatureExtractor.FeatureComparison import FeatureComparison
from ResultData.ComparisonPersistor import ComparisonPersistor

In [6]:
MASK = "[MASK]"
#MASK = "<mask>"

In [21]:
#keys = ["simple","compound","complex","compound-complex", "active", "passive", "nominalized"]
keys = ["simple","compound","complex","compound-complex"]

In [9]:
logging.set_verbosity_error()

In [8]:
model='bert-base-cased'
#model='bert-large-cased'
#model='bert-base-uncased'
#model='bert-large-uncased'
#model='bert-base-multilingual-cased'
#model='bert-base-multilingual-uncased'
#model='roberta-base'
#model='roberta-large'

In [10]:
metric_name = "cosine similarity"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model)

In [12]:
feature_extractor = pipeline("feature-extraction", model=model)

In [ ]:
#relations = get_relations_with_last_digit(3)[2:]+get_relations_with_last_digit(4)+get_relations_with_last_digit(5)+get_relations_with_last_digit(6)+get_relations_with_last_digit(7)+get_relations_with_last_digit(8)+get_relations_with_last_digit(9)
#relations = nominalized_relations

In [ ]:
list(enumerate(relations)), len(relations)

In [14]:
TREx = LamaTRExData(relations = relations)
#TREx = AbstractTRExData(relations=relations)
#TREx = HardKnownTRExData(get_hard_filter, get_known_filter, relations = relations

In [15]:
TREx.load()

In [19]:
persistor = ComparisonPersistor(model, metric_name)

In [17]:
def metric(left, right):
    result = torch.dot(left, right)/(LA.norm(left)*LA.norm(right))
    return result.item()

In [22]:
comparer = FeatureComparison(feature_extractor, tokenizer, metric, relations, get_templates, get_relation_meta, persistor.persist_row_fast, keys=keys, mask=MASK)

In [ ]:
comparer.get_key_pairs()

In [ ]:
comparer.compare(TREx.data)

In [ ]:
persistor.close()

In [ ]:
#persistor.connection.close()

In [24]:
comparer.extract_mask_features(f"Obama was born in {MASK}").shape

torch.Size([768])